## Way1: Python style

In [184]:
# Call packages
import numpy as np
import pandas as pd
import datetime as dt

In [185]:
# Read data
prices = pd.read_csv('prices.csv')
sales = pd.read_csv('sales.csv')

In [186]:
## Standardize datetime type
prices['updated_at'] = pd.to_datetime(prices['updated_at'])
sales['ordered_at'] = pd.to_datetime(pd.to_datetime(sales['ordered_at']).dt.strftime('%Y-%m-%d %H:%M:%S'))

In [187]:
# ### Define function
# Tạo function trả về price trong bảng Prices tương ứng cho mỗi [id, time] trong bảng Sales

# ####  Ý tưởng:
# - Với mỗi id, lấy mốc Price.updated_at lớn nhất nhưng không vượt quá Sales.time, đó sẽ là mốc để lấy 
# Sales.price (= new_price)
# - Nếu không có mốc nào như vậy, tức là Sales.time nhỏ hơn lần update_at đầu tiên, đơn hàng chưa trải 
# qua lần update price nào. Sales.price sẽ là old_price của lần update muộn nhất

In [188]:
## Define function to return price
def return_price(id, time):
## max_update is the latest price-updated-time, which is not later than Sales.time
    max_update = prices[(prices.product_id == id) & (prices.updated_at < time)].updated_at.max()
    
## If existing a max_update like that, the Sales.price will be the new_price of the latest update time, 
## or the Prices.new_price
    get_price = prices[(prices.product_id == id) & (prices.updated_at == max_update)].new_price
    
## If not existing a max_update, means Sales.price hasn't been update yet, the Sales.price will be 
## Prices.old_price of the earliest Prices.update_at
    if get_price.empty == True:
        min_update = prices[(prices.product_id == id)].updated_at.min()
        get_price = prices[(prices.product_id == id) & (prices.updated_at == min_update)].old_price
    
    return(get_price)

In [189]:
# ## Create an array for storing price result
# price_returned_array = pd.array([0]*len(sales))
# for i in sales.index:
#     price_returned_array[i] = return_price(sales.product_id[i], sales.ordered_at[i])

In [190]:
## Create an array for storing price result
price_returned_array = pd.array([]).astype('int64')
for i in sales.index:
    price_returned = return_price(sales.product_id[i], sales.ordered_at[i])
    price_returned_array = np.append(price_returned_array, price_returned)

In [191]:
# Có 1 vấn đề ở đây ạ, type trả về của hàm get_price nó vẫn là Series(object), nếu e dùng list để lưu kết 
# quả của function khi apply vào dataframe thì nó lại ra 1 list of Series(object), e đang phải chuyển 
# thành lưu sang array

# Chạy đoạn này thay vì đoạn dưới:

        # price_returned_list = []
        # for i in sales.index:
        #     price_returned = return_price(sales.product_id[i], sales.ordered_at[i])
        #     price_returned_list.append(price_returned)

        # sales['price'] = price_returned_list
        # sales['revenue'] = sales.quantity_ordered * sales.price

# sẽ ra kết quả sales.price là object chứ k phải int, không pivot được, vậy có cách nào hay hơn k ạ

In [192]:
# E từng thử tạo cột rỗng Sales.price trước, sau đó điền dần data vào:

        # sales['price'] = pd.Series(dtype='int')
        # for i in sales.index:
        #     sales.price[i] = return_price(sales.product_id[i], sales.ordered_at[i])

# thì nó báo warning, nhưng data vẫn gắn được:
#         SettingWithCopyWarning: 
#         A value is trying to be set on a copy of a slice from a DataFrame

# Tức là mình cũng k nên edit trực tiếp lên dataframe như thế mà nên xử lý trên list/array ngoài trước
# sau đó gắn vào sau đúng k ạ?


In [193]:
## Apply price array to df
sales['price'] = price_returned_array
sales['revenue'] = sales.quantity_ordered * sales.price


In [194]:
## Pivot result
sales.pivot_table(
    values= 'revenue',
    index= ['product_id', 'price'],
    aggfunc= sum
)

revenue
product_id price          
64         239000   956000
3954203    57500     57500
           60000    180000
           64000    640000
3998909    15500     15500
           16500    231000
           17000     34000
4085861    52000   1040000
           53500   2140000
           58000   2204000
           60000    180000
           62500   1812500
           67000    871000